In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_azure_ai.chat_models import AzureChatOpenAI

myendpoint = "https://nirav-mkv49s8o-westus3.cognitiveservices.azure.com/"
mydeployment_name = "gpt-4.1"
myapi_version = "2024-12-01-preview"

llm = AzureChatOpenAI(
    model_name=mydeployment_name,
    api_version=myapi_version,
    azure_endpoint=myendpoint
    # api_key=api_key,
)

# llm.invoke([{"role": "user", "content": "Hello, how are you?"}])

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph
# from langgraph.annotations import Annotated
from langgraph.graph.message import add_messages, Annotated

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]} 

builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)

# builder.add_edge("START", "chatbot_node")
# builder.add_edge("chatbot_node", "END")

builder.set_entry_point("chatbot_node")

graph = builder.compile()

In [4]:
messages = {"role": "user", "content": "Hello, how are you?"}
graph.invoke({"messages": [messages]})

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='23455c86-a3d1-49f9-b75f-962811ced6e8'),
  AIMessage(content="Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b9041e10b4', 'id': 'chatcmpl-D52ip7A7GUqw1fZ2rr2wE2m0wSCaQ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'se

In [9]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in ["exit", "quit"]:
        break
    if state is None:
        state = State = {
                            "messages": [{"role": "user", "content": in_message}]
                         }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: The first person to visit (walk on) the Moon was **Neil Armstrong**, an American astronaut. He became the first human to step onto the lunar surface on **July 20, 1969**, during the Apollo 11 mission. His famous words were: *"That's one small step for [a] man, one giant leap for mankind."*
